# Baseline characteristics table - part 2*

### * This code belongs to the paper "Early Prediction of End Stage Kidney Disease Based on Cumulative Estimated Glomerular Filtration Rate Velocity"

In [ ]:
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime

In [ ]:
datapool_ESRD = pd.read_csv('Final_ESRD_group_done_pandas.csv')
full_date_dataset = pd.read_csv('merged_dataset_dates_timedeltas_full.csv')


datapool_ESRD = pd.read_csv('Final_ESRD_group_done_pandas.csv')
datapool_ESRD = datapool_ESRD.drop(columns=datapool_ESRD.columns[0])
datapool_ESRD = datapool_ESRD.drop_duplicates()
datapool_ESRD_dropped = datapool_ESRD.groupby('patient_sk').agg({'eGFR_EPI' : 'count'}).reset_index()[datapool_ESRD.groupby('patient_sk').agg({'eGFR_EPI' : 'count'}).reset_index().eGFR_EPI >=9]
datapool_ESRD_dropped = datapool_ESRD_dropped.drop('eGFR_EPI', axis =1)
datapool_ESRD = datapool_ESRD.merge(datapool_ESRD_dropped, on = 'patient_sk', how = 'inner')

datapool_ESRD['Date'] = pd.to_datetime(datapool_ESRD['Date'])
datapool_ESRD['Date'] = pd.to_datetime(datapool_ESRD['Date'])
datapool_ESRD_dates = datapool_ESRD.groupby('patient_sk').agg({'Date': 'min'})
datapool_ESRD_dates = datapool_ESRD_dates.reset_index()
datapool_ESRD = datapool_ESRD.merge(datapool_ESRD_dates, on = 'patient_sk', how='left')
datapool_ESRD['Date_seconds'] = datapool_ESRD['Date_x'] - datapool_ESRD['Date_y']
datapool_ESRD = datapool_ESRD.rename({'Date_x':'Date'}, axis = 1)
datapool_ESRD = datapool_ESRD.drop('Date_y', axis = 1)
datapool_ESRD['Date_seconds'] = datapool_ESRD['Date_seconds'].dt.total_seconds()

datapool_ESRD_patients = datapool_ESRD.groupby('patient_sk').agg({'Date': 'min'})
datapool_ESRD_patients = datapool_ESRD_patients.reset_index()

datapool_ESRD_patients_eGFR = datapool_ESRD.merge(datapool_ESRD_patients, on=['patient_sk', 'Date'], how ='inner')
datapool_ESRD_patients_eGFR = datapool_ESRD_patients_eGFR.drop_duplicates('patient_sk')
datapool_ESRD_patients_eGFR = datapool_ESRD_patients_eGFR[datapool_ESRD_patients_eGFR['eGFR_EPI']>=60]

datapool_ESRD_new = datapool_ESRD.merge(datapool_ESRD_patients_eGFR['patient_sk'], on = 'patient_sk', how = 'inner')

datapool_ESRD = datapool_ESRD_new
datapool_ESRD = datapool_ESRD.drop_duplicates()

merged_dataset = pd.read_csv('merged_dataset_dates_timedeltas_full.csv')
merged_dataset = merged_dataset.rename(columns={'eGFR_EPI': 'Trigger_eGFR'})
merged_dataset.time_to_event_ESRD = pd.to_timedelta(merged_dataset.time_to_event_ESRD)
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'])
merged_dataset['Diagnosis_admission_date_ESRD'] = pd.to_datetime(merged_dataset['Diagnosis_admission_date_ESRD'])

Urine_Protein_Creatinine_pandas = pd.read_csv('Urine_Protein_Creatinine_pandas.csv')
Urine_Microalbumin_Creatinine_pandas = pd.read_csv('Urine_Microalbumin_Creatinine_pandas.csv')
twentyfour_hr_Urine_Protein_pandas = pd.read_csv('twentyfour_hr_Urine_Protein_pandas.csv')
Urinalysis_Dipstick_Protein_pandas = pd.read_csv('Urinalysis_Dipstick_Protein_pandas.csv')
Hemoglobin_A1c_pandas = pd.read_csv('Hemoglobin_A1c_pandas.csv')
Hemoglobin_pandas = pd.read_csv('Hemoglobin_pandas.csv')
Serum_Calcium_pandas = pd.read_csv('Serum_Calcium_pandas.csv')
Serum_Bicarbonate_pandas = pd.read_csv('Serum_Bicarbonate_pandas.csv')
HIV_pandas = pd.read_csv('HIV_pandas.csv')
Hepatitis_C_pandas = pd.read_csv('Hepatitis_C_pandas.csv')
Serum_Cholesterol_pandas = pd.read_csv('Serum_Cholesterol_pandas.csv')
Serum_albumin_pandas = pd.read_csv('Serum_albumin_pandas.csv')
Serum_Phosphorus_pandas = pd.read_csv('Serum_Phosphorus_pandas.csv')

NSAID_pandas = pd.read_csv('NSAID_pandas.csv')
Proton_Pump_Inhibitors_pandas = pd.read_csv('Proton_Pump_Inhibitors_pandas.csv')
Bipolar_Drugs_pandas = pd.read_csv('Bipolar_Drugs_pandas.csv')

In [ ]:
#for 1 patient, there is no ESRD diagnosis data :(((
merged_dataset.at[3183,'Diagnosis_admission_date_ESRD'] = pd.to_datetime('1990-09-02 12:41:00')
merged_dataset[merged_dataset.patient_sk == 143550162]['Diagnosis_admission_date_ESRD'].values[0]
merged_dataset.time_to_event_ESRD = pd.to_timedelta(merged_dataset.time_to_event_ESRD)

## LOINC CODES

In [ ]:
merged_patient_sk = list(merged_dataset['patient_sk'])
patients_vs_LOINC_lab_procedures = pd.DataFrame({'patient_sk' : merged_patient_sk})

In [ ]:
## Replacing NaN and NaT (pd.null) by 0
pp = []
for i in range(len(list(merged_dataset['patient_sk']))):
   
    if np.isnan(merged_dataset.New_label[i]) == False:
        pp.append(merged_dataset.patient_sk[0])
        
#merged_dataset = merged_dataset.merge(pd.DataFrame({"patient_sk" : pp}), on='patient_sk', how='inner')
trigger_below_20 = list(np.array(list(merged_dataset['patient_sk']))[np.array(list(merged_dataset['Trigger_eGFR']))<20])
print('trigger_below_20:', len(trigger_below_20))

a = np.array(list(merged_dataset['Trigger_eGFR']))<40 
b = np.array(list(merged_dataset['Trigger_eGFR']))>=20
pa = np.array(list(merged_dataset['patient_sk']))[a]
pb = np.array(list(merged_dataset['patient_sk']))[b]                  
trigger_between_20_40 = list(np.intersect1d(pb, pa))
print('trigger_between_20_40:', len(trigger_between_20_40))

a = np.array(list(merged_dataset['Trigger_eGFR']))<60 
b = np.array(list(merged_dataset['Trigger_eGFR']))>=40
pa = np.array(list(merged_dataset['patient_sk']))[a]
pb = np.array(list(merged_dataset['patient_sk']))[b]                  
trigger_between_40_60 = list(np.intersect1d(pb, pa))
print('trigger_between_40_60:', len(trigger_between_40_60))

trigger_above_60 = np.array(list(merged_dataset['patient_sk']))[np.array(list(merged_dataset['Trigger_eGFR']))>=60]
print('trigger_above_60:', len(trigger_above_60))

no_trigger = list(set(list(merged_dataset['patient_sk'])) - (set(trigger_below_20)) - set(trigger_between_20_40) - set(trigger_between_40_60) - set(trigger_above_60))
print('no_trigger:', len(no_trigger))

### Serum_Calcium

In [ ]:
Serum_Calcium_pandas['UOM'].unique()

In [ ]:
dataaa_Serum_Calcium = []
for patient in merged_patient_sk:
    data = Serum_Calcium_pandas.loc[Serum_Calcium_pandas['patient_sk'] == patient]
    data = data.sort_values(by=['Date_Serum_Calcium'])
    data = data.reset_index()
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    data = data.drop('index', axis =1)
    data.Date_Serum_Calcium = pd.to_datetime(data.Date_Serum_Calcium, errors='coerce')
    dataaa_Serum_Calcium.append(data)

In [ ]:
closest_observation_90_days_list = []

i = 0
for patient in dataaa_Serum_Calcium:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_Serum_Calcium'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Serum_Calcium'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'])[i])]['Serum_Calcium']

    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
patients_vs_LOINC_lab_procedures['Serum_Calcium'] = closest_observation_90_days_list

In [ ]:
closest_observation_90_days_list = []

i = 0
for patient in dataaa_Serum_Calcium:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_Serum_Calcium'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Serum_Calcium'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Serum_Calcium']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
np.nanmean(closest_observation_90_days_list)


closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

In [ ]:
closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

In [ ]:
closest_observation_90_days_list = []

i = -1
for patient in dataaa_Serum_Calcium:
    i = i +1
    if patient.patient_sk[0] in no_trigger:
        closest_observation_90_days = 0

        aaa = patient[(patient['Date_Serum_Calcium'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Serum_Calcium'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Serum_Calcium']

        if len(list(aaa)) == 0:
            closest_observation_90_days = np.nan
        else:
            closest_observation_90_days = aaa.iloc[-1]

        closest_observation_90_days_list.append(closest_observation_90_days)

np.nanmean(closest_observation_90_days_list)

closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

In [ ]:
closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

### Urine_Protein_Creatinine_pandas

In [ ]:
Urine_Protein_Creatinine_pandas = pd.read_csv('Urine_Protein_Creatinine_pandas.csv')
Urine_Protein_Creatinine_pandas['UOM'].unique()

In [ ]:
a =  Urine_Protein_Creatinine_pandas[Urine_Protein_Creatinine_pandas['UOM'] == 'mg/g']['Urine_Protein_Creatinine']/1000
Urine_Protein_Creatinine_pandas.loc[Urine_Protein_Creatinine_pandas['UOM'] == 'mg/g', 'Urine_Protein_Creatinine'] = a
Urine_Protein_Creatinine_pandas.loc[Urine_Protein_Creatinine_pandas['UOM'] == 'mg/g', 'UOM'] = 'g/g'


In [ ]:
dataaa_Urine_Protein_Creatinine = []
for patient in merged_patient_sk:
    data = Urine_Protein_Creatinine_pandas.loc[Urine_Protein_Creatinine_pandas['patient_sk'] == patient]
    data = data.sort_values(by=['Date_Urine_Protein_Creatinine'])
    data = data.reset_index()
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    data = data.drop('index', axis =1)
    data.Date_Urine_Protein_Creatinine = pd.to_datetime(data.Date_Urine_Protein_Creatinine, errors='coerce')
    dataaa_Urine_Protein_Creatinine.append(data)

In [ ]:
closest_observation_90_days_list = []

i = 0
for patient in dataaa_Urine_Protein_Creatinine:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_Urine_Protein_Creatinine'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Urine_Protein_Creatinine'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'])[i])]['Urine_Protein_Creatinine']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
patients_vs_LOINC_lab_procedures['Urine_Protein_Creatinine'] = closest_observation_90_days_list



In [ ]:
closest_observation_90_days_list = []

i = 0
for patient in dataaa_Urine_Protein_Creatinine:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_Urine_Protein_Creatinine'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Urine_Protein_Creatinine'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Urine_Protein_Creatinine']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
    
np.nanmean(closest_observation_90_days_list)

closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

In [ ]:
closest_observation_90_days_list = []

i = -1
for patient in dataaa_Urine_Protein_Creatinine:
    i = i +1
    if patient.patient_sk[0] in no_trigger:
        closest_observation_90_days = 0

        aaa = patient[(patient['Date_Urine_Protein_Creatinine'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Urine_Protein_Creatinine'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Urine_Protein_Creatinine']

        if len(list(aaa)) == 0:
            closest_observation_90_days = np.nan
        else:
            closest_observation_90_days = aaa.iloc[-1]
        
        closest_observation_90_days_list.append(closest_observation_90_days)

    
np.nanmean(closest_observation_90_days_list)

closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

### Urine_Microalbumin_Creatinine_pandas NORMALAIZED!

In [ ]:
Urine_Microalbumin_Creatinine_pandas = pd.read_csv('Urine_Microalbumin_Creatinine_pandas_normalized to mg per gram.csv')

In [ ]:
dataaa_Urine_Microalbumin_Creatinine = []
for patient in merged_patient_sk:
    data = Urine_Microalbumin_Creatinine_pandas.loc[Urine_Microalbumin_Creatinine_pandas['patient_sk'] == patient]
    data = data.sort_values(by=['Date_Urine_Microalbumin_Creatinine'])
    data = data.reset_index()
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    data = data.drop('index', axis =1)
    data.Date_Urine_Microalbumin_Creatinine = pd.to_datetime(data.Date_Urine_Microalbumin_Creatinine, errors='coerce')
    dataaa_Urine_Microalbumin_Creatinine.append(data)

In [ ]:
closest_observation_90_days_list = []

i = 0
for patient in dataaa_Urine_Microalbumin_Creatinine:

    aaa = patient[(patient['Date_Urine_Microalbumin_Creatinine'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Urine_Microalbumin_Creatinine'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'])[i])]['Results converted to mg/g']

    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]

    closest_observation_90_days_list.append(closest_observation_90_days)

    i = i +1
patients_vs_LOINC_lab_procedures['Urine_Microalbumin_Creatinine'] = closest_observation_90_days_list

In [ ]:
total_closest_observation_90_days_list = []
trigger_below_20_closest_observation_90_days_list = []            
trigger_between_20_40_closest_observation_90_days_list = []
trigger_between_40_60_closest_observation_90_days_list = []
trigger_above_60_closest_observation_90_days_list = []
no_trigger_closest_observation_90_days_list = []

i = -1
for patient in dataaa_Urine_Microalbumin_Creatinine:
    i = i +1
    closest_observation_90_days = np.nan

    aaa = patient[(patient['Date_Urine_Microalbumin_Creatinine'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Urine_Microalbumin_Creatinine'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'])[i])]['Results converted to mg/g']

    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]

    if (patient.patient_sk[0] in trigger_below_20) & (np.isnan(closest_observation_90_days) == False):
        trigger_below_20_closest_observation_90_days_list.append(closest_observation_90_days)
    elif (patient.patient_sk[0] in trigger_between_20_40) & (np.isnan(closest_observation_90_days) == False):
        trigger_between_20_40_closest_observation_90_days_list.append(closest_observation_90_days)
    elif (patient.patient_sk[0] in trigger_between_40_60) & (np.isnan(closest_observation_90_days) == False):
        trigger_between_40_60_closest_observation_90_days_list.append(closest_observation_90_days)
    elif (patient.patient_sk[0] in trigger_above_60) & (np.isnan(closest_observation_90_days) == False):
        trigger_above_60_closest_observation_90_days_list.append(closest_observation_90_days)
    
    
#now, the total and non triggers

i = -1
for patient in dataaa_Urine_Microalbumin_Creatinine:
    i = i +1
    closest_observation_90_days = np.nan

    aaa = patient[(patient['Date_Urine_Microalbumin_Creatinine'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Urine_Microalbumin_Creatinine'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Results converted to mg/g']

    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
   
    if (np.isnan(closest_observation_90_days) == False):
        total_closest_observation_90_days_list.append(closest_observation_90_days)
        
        
    if (patient.patient_sk[0] in no_trigger) & (np.isnan(closest_observation_90_days) == False):
        no_trigger_closest_observation_90_days_list.append(closest_observation_90_days)

Lisssst = [total_closest_observation_90_days_list,trigger_below_20_closest_observation_90_days_list,trigger_between_20_40_closest_observation_90_days_list,trigger_between_40_60_closest_observation_90_days_list,trigger_above_60_closest_observation_90_days_list,no_trigger_closest_observation_90_days_list]

for item in Lisssst:
    below_30 = []
    between_30_300 = []
    above_300 = []
    
    for thing in item:
        if np.isnan(thing) == False:
            if thing <30:
                below_30.append(thing)
            elif (thing >= 30) & (thing < 300):
                between_30_300.append(thing)
            elif thing >= 300:
                above_300.append(thing)
    print('below_30',len(below_30))
    print('between_30_300',len(between_30_300))
    print('above_300',len(above_300))

### twentyfour_hr_Urine_Protein_pandas

In [ ]:
twentyfour_hr_Urine_Protein_pandas = pd.read_csv('twentyfour_hr_Urine_Protein_pandas.csv')

In [ ]:
twentyfour_hr_Urine_Protein_pandas['UOM'].unique()

In [ ]:
twentyfour_hr_Urine_Protein_pandas.loc[twentyfour_hr_Urine_Protein_pandas['UOM'] == 'mg/dL', '24_hr_Urine_Protein'] = np.nan
twentyfour_hr_Urine_Protein_pandas.loc[twentyfour_hr_Urine_Protein_pandas['UOM'] == 'mL/day', '24_hr_Urine_Protein'] = np.nan
twentyfour_hr_Urine_Protein_pandas.loc[twentyfour_hr_Urine_Protein_pandas['UOM'] == 'mL', '24_hr_Urine_Protein'] = np.nan
twentyfour_hr_Urine_Protein_pandas.loc[pd.isnull(twentyfour_hr_Urine_Protein_pandas['UOM']) == True, '24_hr_Urine_Protein'] = np.nan

In [ ]:
dataaa_twentyfour_hr_Urine_Protein = []
for patient in merged_patient_sk:
    data = twentyfour_hr_Urine_Protein_pandas.loc[twentyfour_hr_Urine_Protein_pandas['patient_sk'] == patient]
    data = data.sort_values(by=['Date_24_hr_Urine_Protein'])
    data = data.reset_index()
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    data = data.drop('index', axis =1)
    data.Date_24_hr_Urine_Protein = pd.to_datetime(data.Date_24_hr_Urine_Protein, errors='coerce')
    dataaa_twentyfour_hr_Urine_Protein.append(data)

In [ ]:
closest_observation_90_days_list = []

i = 0
for patient in dataaa_twentyfour_hr_Urine_Protein:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_24_hr_Urine_Protein'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_24_hr_Urine_Protein'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'])[i])]['24_hr_Urine_Protein']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
patients_vs_LOINC_lab_procedures['24_hr_Urine_Protein'] = closest_observation_90_days_list


In [ ]:

closest_observation_90_days_list = []

i = 0
for patient in dataaa_twentyfour_hr_Urine_Protein:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_24_hr_Urine_Protein'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_24_hr_Urine_Protein'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['24_hr_Urine_Protein']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
    
np.nanmean(closest_observation_90_days_list)

closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

In [ ]:
closest_observation_90_days_list = []

i = -1
for patient in dataaa_twentyfour_hr_Urine_Protein:
    i = i +1
    if patient.patient_sk[0] in no_trigger:
        closest_observation_90_days = 0

        aaa = patient[(patient['Date_24_hr_Urine_Protein'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_24_hr_Urine_Protein'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['24_hr_Urine_Protein']

        if len(list(aaa)) == 0:
            closest_observation_90_days = np.nan
        else:
            closest_observation_90_days = aaa.iloc[-1]
        
        closest_observation_90_days_list.append(closest_observation_90_days)

    
np.nanmean(closest_observation_90_days_list)

closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

### Hemoglobin_A1c_pandas  NONE!

In [ ]:
Hemoglobin_A1c_pandas = pd.read_csv('Hemoglobin_A1c_pandas.csv')

In [ ]:
a =  Hemoglobin_A1c_pandas[Hemoglobin_A1c_pandas['UOM'] == '%']['SBP_Hemoglobin_A1c']
Hemoglobin_A1c_pandas.loc[Hemoglobin_A1c_pandas['UOM'] == '%', 'SBP_Hemoglobin_A1c'] = a
Hemoglobin_A1c_pandas.loc[Hemoglobin_A1c_pandas['UOM'] == '%', 'UOM'] = '%'

a =  Hemoglobin_A1c_pandas[Hemoglobin_A1c_pandas['UOM'] == '/100']['SBP_Hemoglobin_A1c']
Hemoglobin_A1c_pandas.loc[Hemoglobin_A1c_pandas['UOM'] == '/100', 'SBP_Hemoglobin_A1c'] = a
Hemoglobin_A1c_pandas.loc[Hemoglobin_A1c_pandas['UOM'] == '/100', 'UOM'] = '%'


Hemoglobin_A1c_pandas.loc[Hemoglobin_A1c_pandas['SBP_Hemoglobin_A1c'] < 4, 'SBP_Hemoglobin_A1c'] = np.nan

Hemoglobin_A1c_pandas.loc[(Hemoglobin_A1c_pandas['UOM'] == 'Not Mapped') | (Hemoglobin_A1c_pandas['UOM'] =='umol/L') | (Hemoglobin_A1c_pandas['UOM'] =='mmol/L') , 'SBP_Hemoglobin_A1c'] = np.nan
Hemoglobin_A1c_pandas.loc[pd.isnull(Hemoglobin_A1c_pandas['UOM']) == True, 'SBP_Hemoglobin_A1c'] = np.nan

In [ ]:
dataaa_Hemoglobin_A1c = []
for patient in merged_patient_sk:
    data = Hemoglobin_A1c_pandas.loc[Hemoglobin_A1c_pandas['patient_sk'] == patient]
    data = data.sort_values(by=['Date_Hemoglobin_A1c'])
    data = data.reset_index()
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    data = data.drop('index', axis =1)
    data.Date_Hemoglobin_A1c = pd.to_datetime(data.Date_Hemoglobin_A1c, errors='coerce')
    dataaa_Hemoglobin_A1c.append(data)    

In [ ]:
closest_observation_90_days_list = []

i = 0
for patient in dataaa_Hemoglobin_A1c:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_Hemoglobin_A1c'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Hemoglobin_A1c'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'])[i])]['SBP_Hemoglobin_A1c']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
patients_vs_LOINC_lab_procedures['Hemoglobin_A1c'] = closest_observation_90_days_list


In [ ]:

closest_observation_90_days_list = []

i = 0
for patient in dataaa_Hemoglobin_A1c:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_Hemoglobin_A1c'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Hemoglobin_A1c'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['SBP_Hemoglobin_A1c']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
    
np.nanmean(closest_observation_90_days_list)

closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

In [ ]:

closest_observation_90_days_list = []

i = -1
for patient in dataaa_Hemoglobin_A1c:
    i = i +1
    if patient.patient_sk[0] in no_trigger:
        closest_observation_90_days = 0

        aaa = patient[(patient['Date_Hemoglobin_A1c'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Hemoglobin_A1c'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['SBP_Hemoglobin_A1c']

        if len(list(aaa)) == 0:
            closest_observation_90_days = np.nan
        else:
            closest_observation_90_days = aaa.iloc[-1]
        
        closest_observation_90_days_list.append(closest_observation_90_days)

    
np.nanmean(closest_observation_90_days_list)

closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

### Hemoglobin

In [ ]:
Hemoglobin_pandas['UOM'].unique()
Hemoglobin_pandas=pd.read_csv('Hemoglobin_pandas.csv')

In [ ]:
a =  Hemoglobin_pandas[Hemoglobin_pandas['UOM'] == '%']['Hemoglobin']
Hemoglobin_pandas.loc[Hemoglobin_pandas['UOM'] == '%', 'Hemoglobin'] = a
Hemoglobin_pandas.loc[Hemoglobin_pandas['UOM'] == '%', 'UOM'] = 'g/dL'

a =  Hemoglobin_pandas[Hemoglobin_pandas['UOM'] == 'g/mL']['Hemoglobin']
Hemoglobin_pandas.loc[Hemoglobin_pandas['UOM'] == 'g/mL', 'Hemoglobin'] = a
Hemoglobin_pandas.loc[Hemoglobin_pandas['UOM'] == 'g/mL', 'UOM'] = 'g/dL'

a =  Hemoglobin_pandas[Hemoglobin_pandas['UOM'] == 'mg/dL']['Hemoglobin']*0.001
Hemoglobin_pandas.loc[Hemoglobin_pandas['UOM'] == 'mg/dL', 'Hemoglobin'] = np.nan
Hemoglobin_pandas.loc[Hemoglobin_pandas['UOM'] == 'mg/dL', 'UOM'] = np.nan

Hemoglobin_pandas.loc[Hemoglobin_pandas['UOM'] == 'g', 'UOM'] = 'g/dL'

Hemoglobin_pandas.loc[Hemoglobin_pandas['UOM'] == 'Not Mapped', 'Hemoglobin'] = np.nan
Hemoglobin_pandas.loc[pd.isnull(Hemoglobin_pandas['UOM']) == True, 'Hemoglobin'] = np.nan

In [ ]:
dataaa_Hemoglobin = []
for patient in merged_patient_sk:
    data = Hemoglobin_pandas.loc[Hemoglobin_pandas['patient_sk'] == patient]
    data = data.sort_values(by=['Date_Hemoglobin'])
    data = data.reset_index()
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    data = data.drop('index', axis =1)
    data.Date_Hemoglobin = pd.to_datetime(data.Date_Hemoglobin, errors='coerce')
    dataaa_Hemoglobin.append(data)
  

In [ ]:

closest_observation_90_days_list = []

i = 0
for patient in dataaa_Hemoglobin:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_Hemoglobin'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Hemoglobin'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'])[i])]['Hemoglobin']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
patients_vs_LOINC_lab_procedures['Hemoglobin'] = closest_observation_90_days_list


In [ ]:

closest_observation_90_days_list = []

i = 0
for patient in dataaa_Hemoglobin:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_Hemoglobin'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Hemoglobin'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Hemoglobin']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
    
np.nanmean(closest_observation_90_days_list)

closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

In [ ]:

closest_observation_90_days_list = []

i = -1
for patient in dataaa_Hemoglobin:
    i = i +1
    if patient.patient_sk[0] in no_trigger:
        closest_observation_90_days = 0

        aaa = patient[(patient['Date_Hemoglobin'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Hemoglobin'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Hemoglobin']

        if len(list(aaa)) == 0:
            closest_observation_90_days = np.nan
        else:
            closest_observation_90_days = aaa.iloc[-1]
        
        closest_observation_90_days_list.append(closest_observation_90_days)

    
np.nanmean(closest_observation_90_days_list)


closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

###  Hepatitis_C_Positive

In [ ]:
Hepatitis_C_pandas = pd.read_csv('Hepatitis_C_pandas.csv')
Hepatitis_C_pandas['UOM'].unique()

In [ ]:
Hepatitis_C_pandas[pd.isnull(Hepatitis_C_pandas.Date_Hepatitis_C) == False]['patient_sk'].unique().shape[0]

In [ ]:
# Habing Hepatitis C EVER before the trigger date

dataaa_Hepatitis_C = []
for patient in merged_patient_sk:
    data = Hepatitis_C_pandas.loc[Hepatitis_C_pandas['patient_sk'] == patient]
    data = data.sort_values(by=['Date_Hepatitis_C'])
    data = data.reset_index()
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    data = data.drop('index', axis =1)
    data.Date_Hepatitis_C = pd.to_datetime(data.Date_Hepatitis_C, errors='coerce')
    dataaa_Hepatitis_C.append(data)

In [ ]:
i = 0
avg_1month = 0
aaa =[]
average_list = []
for patient in dataaa_Hepatitis_C:

    aaa = patient[patient['Date_Hepatitis_C'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'])[i]]['Hepatitis_C']

    if len(list(aaa)) == 0:
        avg_1month = np.nan
    else:
        avg_1month = 1

    average_list.append(avg_1month)
    i = i +1
patients_vs_LOINC_lab_procedures['Hepatitis_C_1month'] = average_list    

In [ ]:
average_list = []


i = -1
for patient in dataaa_Hepatitis_C:
    i = i +1
    if patient.patient_sk[0] in no_trigger:
        avg_1month = 0

        aaa = patient[(patient['Date_Hepatitis_C'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Hepatitis_C']

        if len(list(aaa)) == 0:
            avg_1month = np.nan
        else:
            avg_1month = 1

        average_list.append(avg_1month)
    

In [ ]:
count = 0
for i in average_list:
    if np.isnan(i) == False:
        count = count + 1
count

###  Serum_Cholesterol

In [ ]:
Serum_Cholesterol_pandas['UOM'].unique()

In [ ]:
Serum_Cholesterol_pandas.loc[Serum_Cholesterol_pandas['UOM'] == 'Not Mapped', 'Serum_Cholesterol'] = np.nan
Serum_Cholesterol_pandas.loc[pd.isnull(Serum_Cholesterol_pandas['UOM']) == True, 'Serum_Cholesterol'] = np.nan

In [ ]:
dataaa_Serum_Cholesterol = []
for patient in merged_patient_sk:
    data = Serum_Cholesterol_pandas.loc[Serum_Cholesterol_pandas['patient_sk'] == patient]
    data = data.sort_values(by=['Date_Serum_Cholesterol'])
    data = data.reset_index()
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    data = data.drop('index', axis =1)
    data.Date_Serum_Cholesterol = pd.to_datetime(data.Date_Serum_Cholesterol, errors='coerce')
    dataaa_Serum_Cholesterol.append(data)

In [ ]:

closest_observation_90_days_list = []

i = 0
for patient in dataaa_Serum_Cholesterol:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_Serum_Cholesterol'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Serum_Cholesterol'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'])[i])]['Serum_Cholesterol']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
patients_vs_LOINC_lab_procedures['Serum_Cholesterol'] = closest_observation_90_days_list


In [ ]:

closest_observation_90_days_list = []

i = 0
for patient in dataaa_Serum_Cholesterol:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_Serum_Cholesterol'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Serum_Cholesterol'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Serum_Cholesterol']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
    
np.nanmean(closest_observation_90_days_list)


closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

In [ ]:

closest_observation_90_days_list = []

i = -1
for patient in dataaa_Serum_Cholesterol:
    i = i +1
    if patient.patient_sk[0] in no_trigger:
        closest_observation_90_days = 0

        aaa = patient[(patient['Date_Serum_Cholesterol'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Serum_Cholesterol'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Serum_Cholesterol']

        if len(list(aaa)) == 0:
            closest_observation_90_days = np.nan
        else:
            closest_observation_90_days = aaa.iloc[-1]
        
        closest_observation_90_days_list.append(closest_observation_90_days)

    
np.nanmean(closest_observation_90_days_list)


closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

###  Serum_albumin_pandas

In [ ]:
Serum_albumin_pandas = pd.read_csv('Serum_albumin_pandas.csv')

In [ ]:
Serum_albumin_pandas['UOM'].unique()

In [ ]:
Serum_albumin_pandas[Serum_albumin_pandas['UOM'] == '%']

In [ ]:
a =  Serum_albumin_pandas[Serum_albumin_pandas['UOM'] == '%']['Serum_albumin']*0.1
Serum_albumin_pandas.loc[Serum_albumin_pandas['UOM'] == '%', 'Serum_albumin'] = a
Serum_albumin_pandas.loc[Serum_albumin_pandas['UOM'] == '%', 'UOM'] = 'g/dL'

a =  Serum_albumin_pandas[Serum_albumin_pandas['UOM'] == 'mg/dL']['Serum_albumin']
Serum_albumin_pandas.loc[Serum_albumin_pandas['UOM'] == 'mg/dL', 'Serum_albumin'] = a
Serum_albumin_pandas.loc[Serum_albumin_pandas['UOM'] == 'mg/dL', 'UOM'] = 'g/dL'

Serum_albumin_pandas.loc[pd.isnull(Serum_albumin_pandas['UOM']) == True, 'Serum_albumin'] = np.nan
Serum_albumin_pandas.loc[Serum_albumin_pandas['UOM'] == 'mg/24h', 'Serum_albumin'] = np.nan
Serum_albumin_pandas.loc[Serum_albumin_pandas['Serum_albumin'] >= 100, 'Serum_albumin'] = np.nan

Serum_albumin_pandas.loc[Serum_albumin_pandas['UOM'] == 'Not Mapped', 'UOM'] = 'g/dL'

In [ ]:
dataaa_Serum_albumin = []
for patient in merged_patient_sk:
    data = Serum_albumin_pandas.loc[Serum_albumin_pandas['patient_sk'] == patient]
    data = data.sort_values(by=['Date_Serum_albumin'])
    data = data.reset_index()
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    data = data.drop('index', axis =1)
    data.Date_Serum_albumin = pd.to_datetime(data.Date_Serum_albumin, errors='coerce')
    dataaa_Serum_albumin.append(data)

In [ ]:

closest_observation_90_days_list = []

i = 0
for patient in dataaa_Serum_albumin:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_Serum_albumin'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Serum_albumin'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'])[i])]['Serum_albumin']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
patients_vs_LOINC_lab_procedures['Serum_albumin'] = closest_observation_90_days_list


In [ ]:

closest_observation_90_days_list = []

i = 0
for patient in dataaa_Serum_albumin:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_Serum_albumin'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Serum_albumin'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Serum_albumin']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
    
np.nanmean(closest_observation_90_days_list)


closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

In [ ]:

closest_observation_90_days_list = []

i = -1
for patient in dataaa_Serum_albumin:
    i = i +1
    if patient.patient_sk[0] in no_trigger:
        closest_observation_90_days = 0

        aaa = patient[(patient['Date_Serum_albumin'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Serum_albumin'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Serum_albumin']

        if len(list(aaa)) == 0:
            closest_observation_90_days = np.nan
        else:
            closest_observation_90_days = aaa.iloc[-1]
        
        closest_observation_90_days_list.append(closest_observation_90_days)

    
np.nanmean(closest_observation_90_days_list)


closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

###  Serum_Phosphorus_pandas

In [ ]:
Serum_Phosphorus_pandas['UOM'].unique()

In [ ]:
Serum_Phosphorus_pandas.loc[pd.isnull(Serum_Phosphorus_pandas['UOM']) == True, 'Serum_Phosphorus'] = np.nan
Serum_Phosphorus_pandas.loc[Serum_Phosphorus_pandas['UOM'] == 'Not Mapped', 'Serum_Phosphorus'] = np.nan

In [ ]:
dataaa_Serum_Phosphorus = []
for patient in merged_patient_sk:
    data = Serum_Phosphorus_pandas.loc[Serum_Phosphorus_pandas['patient_sk'] == patient]
    data = data.sort_values(by=['Date_Serum_Phosphorus'])
    data = data.reset_index()
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    data = data.drop('index', axis =1)
    data.Date_Serum_Phosphorus = pd.to_datetime(data.Date_Serum_Phosphorus, errors='coerce')
    dataaa_Serum_Phosphorus.append(data)

In [ ]:

closest_observation_90_days_list = []

i = 0
for patient in dataaa_Serum_Phosphorus:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_Serum_Phosphorus'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Serum_Phosphorus'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'])[i])]['Serum_Phosphorus']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
patients_vs_LOINC_lab_procedures['Serum_Phosphorus'] = closest_observation_90_days_list


In [ ]:

closest_observation_90_days_list = []

i = 0
for patient in dataaa_Serum_Phosphorus:
    closest_observation_90_days = 0
    
    aaa = patient[(patient['Date_Serum_Phosphorus'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Serum_Phosphorus'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Serum_Phosphorus']
        
    if len(list(aaa)) == 0:
        closest_observation_90_days = np.nan
    else:
        closest_observation_90_days = aaa.iloc[-1]
        
    closest_observation_90_days_list.append(closest_observation_90_days)
    i = i +1
    
np.nanmean(closest_observation_90_days_list)


closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

In [ ]:

closest_observation_90_days_list = []

i = -1
for patient in dataaa_Serum_Phosphorus:
    i = i +1
    if patient.patient_sk[0] in no_trigger:
        closest_observation_90_days = 0

        aaa = patient[(patient['Date_Serum_Phosphorus'] >= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'] - datetime.timedelta(seconds=90*86400))[i]) & (patient['Date_Serum_Phosphorus'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Serum_Phosphorus']

        if len(list(aaa)) == 0:
            closest_observation_90_days = np.nan
        else:
            closest_observation_90_days = aaa.iloc[-1]
        
        closest_observation_90_days_list.append(closest_observation_90_days)

    
np.nanmean(closest_observation_90_days_list)


closest_observation_90_days_list_not_nan = [x for x in closest_observation_90_days_list if np.isnan(x)==False]
print('closest_observation_90_days_list_mean = ', np.mean(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_count = ', len(closest_observation_90_days_list_not_nan))
print('closest_observation_90_days_list_sd = ', np.sum(np.power(closest_observation_90_days_list_not_nan, 2)))

In [ ]:
patients_vs_LOINC_lab_procedures

# MEDICATIONS

In [ ]:
merged_dataset['Trigger_date'] = pd.to_datetime(merged_dataset['Trigger_date'], errors='coerce')
merged_patient_sk = list(merged_dataset['patient_sk'])
patients_vs_medications = pd.DataFrame({'patient_sk' : merged_patient_sk})

In [ ]:
NSAID_pandas[pd.isnull(NSAID_pandas.Date_NSAID) == False]['patient_sk'].unique().shape[0]

In [ ]:
dataaa_NSAID = []
for patient in merged_patient_sk:
    data = NSAID_pandas.loc[NSAID_pandas['patient_sk'] == patient]
    data = data.sort_values(by=['Date_NSAID'])
    data = data.reset_index()
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    data = data.drop('index', axis =1)
    data.Date_NSAID = pd.to_datetime(data.Date_NSAID, errors='coerce')
    dataaa_NSAID.append(data)
    
    
average_list = []

i = 0
for patient in dataaa_NSAID:
    avg_1month = 0
    
    aaa = patient[patient['Date_NSAID'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'])[i]]['Date_NSAID']
    
    if len(list(aaa)) == 0:
        avg_1month = np.nan
    else:
        avg_1month = 1
        
    average_list.append(avg_1month)
    i = i +1
patients_vs_medications['NSAID'] = average_list    

In [ ]:
average_list = []


i = -1
for patient in dataaa_NSAID:
    i = i +1
    if patient.patient_sk[0] in no_trigger:
        avg_1month = 0

        aaa = patient[(patient['Date_NSAID'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Date_NSAID']

        if len(list(aaa)) == 0:
            avg_1month = np.nan
        else:
            avg_1month = 1

        average_list.append(avg_1month)

count = 0
for i in average_list:
    if np.isnan(i) == False:
        count = count + 1
count

In [ ]:
Proton_Pump_Inhibitors_pandas[pd.isnull(Proton_Pump_Inhibitors_pandas.Date_Proton_Pump_Inhibitors) == False]['patient_sk'].unique().shape[0]

In [ ]:
dataaa_Proton_Pump_Inhibitors = []
for patient in merged_patient_sk:
    data = Proton_Pump_Inhibitors_pandas.loc[Proton_Pump_Inhibitors_pandas['patient_sk'] == patient]
    data = data.sort_values(by=['Date_Proton_Pump_Inhibitors'])
    data = data.reset_index()
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    data = data.drop('index', axis =1)
    data.Date_Proton_Pump_Inhibitors = pd.to_datetime(data.Date_Proton_Pump_Inhibitors, errors='coerce')
    dataaa_Proton_Pump_Inhibitors.append(data)
    
    
average_list = []

i = 0
for patient in dataaa_Proton_Pump_Inhibitors:
    avg_1month = 0
    
    aaa = patient[patient['Date_Proton_Pump_Inhibitors'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'])[i]]['Date_Proton_Pump_Inhibitors']
    
    if len(list(aaa)) == 0:
        avg_1month = None
    else:
        avg_1month = 1
        
    average_list.append(avg_1month)
    i = i +1
patients_vs_medications['Proton_Pump_Inhibitors'] = average_list    

In [ ]:
average_list = []


i = -1
for patient in dataaa_Proton_Pump_Inhibitors:
    i = i +1
    if patient.patient_sk[0] in no_trigger:
        avg_1month = 0

        aaa = patient[(patient['Date_Proton_Pump_Inhibitors'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Date_Proton_Pump_Inhibitors']

        if len(list(aaa)) == 0:
            avg_1month = np.nan
        else:
            avg_1month = 1

        average_list.append(avg_1month)

count = 0
for i in average_list:
    if np.isnan(i) == False:
        count = count + 1
count

In [ ]:
Bipolar_Drugs_pandas[pd.isnull(Bipolar_Drugs_pandas.Date_Bipolar_Drugs) == False]['patient_sk'].unique().shape[0]

In [ ]:
dataaa_Bipolar_Drugs = []
for patient in merged_patient_sk:
    data = Bipolar_Drugs_pandas.loc[Bipolar_Drugs_pandas['patient_sk'] == patient]
    data = data.sort_values(by=['Date_Bipolar_Drugs'])
    data = data.reset_index()
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    data = data.drop('index', axis =1)
    data.Date_Bipolar_Drugs = pd.to_datetime(data.Date_Bipolar_Drugs, errors='coerce')
    dataaa_Bipolar_Drugs.append(data)
    
    
average_list = []

i = 0
for patient in dataaa_Bipolar_Drugs:
    avg_1month = 0
    
    aaa = patient[patient['Date_Bipolar_Drugs'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Trigger_date'])[i]]['Date_Bipolar_Drugs']
    
    if len(list(aaa)) == 0:
        avg_1month = None
    else:
        avg_1month = 1
        
    average_list.append(avg_1month)
    i = i +1
patients_vs_medications['Bipolar_Drugs'] = average_list    

In [ ]:
average_list = []


i = -1
for patient in dataaa_Bipolar_Drugs:
    i = i +1
    if patient.patient_sk[0] in no_trigger:
        avg_1month = 0

        aaa = patient[(patient['Date_Bipolar_Drugs'] <= (merged_dataset[merged_dataset['patient_sk']==patient['patient_sk'][0]]['Diagnosis_admission_date_ESRD'])[i])]['Date_Bipolar_Drugs']

        if len(list(aaa)) == 0:
            avg_1month = np.nan
        else:
            avg_1month = 1

        average_list.append(avg_1month)

count = 0
for i in average_list:
    if np.isnan(i) == False:
        count = count + 1
count

In [ ]:
patients_vs_medications

# Okay Now, The final calculations

In [ ]:
## Replacing NaN and NaT (pd.null) by 0
pp = []
for i in range(len(list(merged_dataset['patient_sk']))):
   
    if np.isnan(merged_dataset.New_label[i]) == False:
        pp.append(merged_dataset.patient_sk[0])
        
#merged_dataset = merged_dataset.merge(pd.DataFrame({"patient_sk" : pp}), on='patient_sk', how='inner')
trigger_below_20 = list(np.array(list(merged_dataset['patient_sk']))[np.array(list(merged_dataset['Trigger_eGFR']))<20])
print('trigger_below_20:', len(trigger_below_20))

a = np.array(list(merged_dataset['Trigger_eGFR']))<40 
b = np.array(list(merged_dataset['Trigger_eGFR']))>=20
pa = np.array(list(merged_dataset['patient_sk']))[a]
pb = np.array(list(merged_dataset['patient_sk']))[b]                  
trigger_between_20_40 = list(np.intersect1d(pb, pa))
print('trigger_between_20_40:', len(trigger_between_20_40))

a = np.array(list(merged_dataset['Trigger_eGFR']))<60 
b = np.array(list(merged_dataset['Trigger_eGFR']))>=40
pa = np.array(list(merged_dataset['patient_sk']))[a]
pb = np.array(list(merged_dataset['patient_sk']))[b]                  
trigger_between_40_60 = list(np.intersect1d(pb, pa))
print('trigger_between_40_60:', len(trigger_between_40_60))

trigger_above_60 = np.array(list(merged_dataset['patient_sk']))[np.array(list(merged_dataset['Trigger_eGFR']))>=60]
print('trigger_above_60:', len(trigger_above_60))

no_trigger = list(set(list(merged_dataset['patient_sk'])) - (set(trigger_below_20)) - set(trigger_between_20_40) - set(trigger_between_40_60) - set(trigger_above_60))
print('no_trigger:', len(no_trigger))

In [ ]:
# Urine_Microalbumin_Creatinine

count_list = list(patients_vs_LOINC_lab_procedures['patient_sk'])
item_list = list(patients_vs_LOINC_lab_procedures['Urine_Microalbumin_Creatinine'])  

trigger_below_20 = list(set(count_list) & set(trigger_below_20))
trigger_between_20_40 = list(set(count_list) & set(trigger_between_20_40))
trigger_between_40_60 = list(set(count_list) & set(trigger_between_40_60))
trigger_above_60 = list(set(count_list) & set(trigger_above_60))

trigger_below_20_Urine_Microalbumin_Creatinine_mean = []
trigger_between_20_40_Urine_Microalbumin_Creatinine_mean = []
trigger_between_40_60_Urine_Microalbumin_Creatinine_mean = []
trigger_above_60_Urine_Microalbumin_Creatinine_mean = []

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        indexx = count_list.index(patient_sk)
        trigger_below_20_Urine_Microalbumin_Creatinine_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_20_40:
        indexx = count_list.index(patient_sk)
        trigger_between_20_40_Urine_Microalbumin_Creatinine_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_40_60:
        indexx = count_list.index(patient_sk)
        trigger_between_40_60_Urine_Microalbumin_Creatinine_mean.append(item_list[indexx])
    elif patient_sk in trigger_above_60:
        indexx = count_list.index(patient_sk)
        trigger_above_60_Urine_Microalbumin_Creatinine_mean.append(item_list[indexx])

trigger_below_20_Urine_Microalbumin_Creatinine_mean_not_nan = [x for x in trigger_below_20_Urine_Microalbumin_Creatinine_mean if np.isnan(x)==False]
trigger_between_20_40_Urine_Microalbumin_Creatinine_mean_not_nan = [x for x in trigger_between_20_40_Urine_Microalbumin_Creatinine_mean if np.isnan(x)==False]
trigger_between_40_60_Urine_Microalbumin_Creatinine_mean_not_nan = [x for x in trigger_between_40_60_Urine_Microalbumin_Creatinine_mean if np.isnan(x)==False]
trigger_above_60_Urine_Microalbumin_Creatinine_mean_not_nan = [x for x in trigger_above_60_Urine_Microalbumin_Creatinine_mean if np.isnan(x)==False]

print('trigger_below_20_Urine_Microalbumin_Creatinine_mean = ', np.nanmean(trigger_below_20_Urine_Microalbumin_Creatinine_mean_not_nan))
print('trigger_between_20_40_Urine_Microalbumin_Creatinine_mean = ', np.nanmean(trigger_between_20_40_Urine_Microalbumin_Creatinine_mean_not_nan))
print('trigger_between_40_60_Urine_Microalbumin_Creatinine_mean = ', np.nanmean(trigger_between_40_60_Urine_Microalbumin_Creatinine_mean_not_nan))
print('trigger_above_60_Urine_Microalbumin_Creatinine_mean = ', np.nanmean(trigger_above_60_Urine_Microalbumin_Creatinine_mean_not_nan))

print('trigger_below_20_Urine_Microalbumin_Creatinine_count = ', len(trigger_below_20_Urine_Microalbumin_Creatinine_mean_not_nan))
print('trigger_between_20_40_Urine_Microalbumin_Creatinine_count = ', len(trigger_between_20_40_Urine_Microalbumin_Creatinine_mean_not_nan))
print('trigger_between_40_60_Urine_Microalbumin_Creatinine_count = ', len(trigger_between_40_60_Urine_Microalbumin_Creatinine_mean_not_nan))
print('trigger_above_60_Urine_Microalbumin_Creatinine_count = ', len(trigger_above_60_Urine_Microalbumin_Creatinine_mean_not_nan))

print('trigger_below_20_Urine_Microalbumin_Creatinine_sd = ', np.sum(np.power(trigger_below_20_Urine_Microalbumin_Creatinine_mean_not_nan, 2)))
print('trigger_between_20_40_Urine_Microalbumin_Creatinine_sd = ', np.sum(np.power(trigger_between_20_40_Urine_Microalbumin_Creatinine_mean_not_nan, 2)))
print('trigger_between_40_60_Urine_Microalbumin_Creatinine_sd = ', np.sum(np.power(trigger_between_40_60_Urine_Microalbumin_Creatinine_mean_not_nan, 2)))
print('trigger_above_60_Urine_Microalbumin_Creatinine_sd = ', np.sum(np.power(trigger_above_60_Urine_Microalbumin_Creatinine_mean_not_nan, 2)))


In [ ]:
# Serum_Calcium

count_list = list(patients_vs_LOINC_lab_procedures['patient_sk'])
item_list = list(patients_vs_LOINC_lab_procedures['Serum_Calcium'])

trigger_below_20 = list(set(count_list) & set(trigger_below_20))
trigger_between_20_40 = list(set(count_list) & set(trigger_between_20_40))
trigger_between_40_60 = list(set(count_list) & set(trigger_between_40_60))
trigger_above_60 = list(set(count_list) & set(trigger_above_60))

trigger_below_20_Serum_Calcium_mean = []
trigger_between_20_40_Serum_Calcium_mean = []
trigger_between_40_60_Serum_Calcium_mean = []
trigger_above_60_Serum_Calcium_mean = []

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        indexx = count_list.index(patient_sk)
        trigger_below_20_Serum_Calcium_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_20_40:
        indexx = count_list.index(patient_sk)
        trigger_between_20_40_Serum_Calcium_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_40_60:
        indexx = count_list.index(patient_sk)
        trigger_between_40_60_Serum_Calcium_mean.append(item_list[indexx])
    elif patient_sk in trigger_above_60:
        indexx = count_list.index(patient_sk)
        trigger_above_60_Serum_Calcium_mean.append(item_list[indexx])

print('trigger_below_20_Serum_Calcium_mean = ', np.nanmean(trigger_below_20_Serum_Calcium_mean))
print('trigger_between_20_40_Serum_Calcium_mean = ', np.nanmean(trigger_between_20_40_Serum_Calcium_mean))
print('trigger_between_40_60_Serum_Calcium_mean = ', np.nanmean(trigger_between_40_60_Serum_Calcium_mean))
print('trigger_above_60_Serum_Calcium_mean = ', np.nanmean(trigger_above_60_Serum_Calcium_mean))


trigger_below_20_Serum_Calcium_mean_not_nan = [x for x in trigger_below_20_Serum_Calcium_mean if np.isnan(x)==False]
trigger_between_20_40_Serum_Calcium_mean_not_nan = [x for x in trigger_between_20_40_Serum_Calcium_mean if np.isnan(x)==False]
trigger_between_40_60_Serum_Calcium_mean_not_nan = [x for x in trigger_between_40_60_Serum_Calcium_mean if np.isnan(x)==False]
trigger_above_60_Serum_Calcium_mean_not_nan = [x for x in trigger_above_60_Serum_Calcium_mean if np.isnan(x)==False]

print('trigger_below_20_Serum_Calcium_mean = ', np.nanmean(trigger_below_20_Serum_Calcium_mean_not_nan))
print('trigger_between_20_40_Serum_Calcium_mean = ', np.nanmean(trigger_between_20_40_Serum_Calcium_mean_not_nan))
print('trigger_between_40_60_Serum_Calcium_mean = ', np.nanmean(trigger_between_40_60_Serum_Calcium_mean_not_nan))
print('trigger_above_60_Serum_Calcium_mean = ', np.nanmean(trigger_above_60_Serum_Calcium_mean_not_nan))

print('trigger_below_20_Serum_Calcium_mean_count = ', len(trigger_below_20_Serum_Calcium_mean_not_nan))
print('trigger_between_20_40_Serum_Calcium_mean_count = ', len(trigger_between_20_40_Serum_Calcium_mean_not_nan))
print('trigger_between_20_40_Serum_Calcium_mean_count = ', len(trigger_between_40_60_Serum_Calcium_mean_not_nan))
print('trigger_above_60_Serum_Calcium_mean_count = ', len(trigger_above_60_Serum_Calcium_mean_not_nan))

print('trigger_below_20_Serum_Calcium_mean_sd = ', np.sum(np.power(trigger_below_20_Serum_Calcium_mean_not_nan, 2)))
print('trigger_between_20_40_Serum_Calcium_mean_sd = ', np.sum(np.power(trigger_between_20_40_Serum_Calcium_mean_not_nan, 2)))
print('trigger_between_20_40_Serum_Calcium_mean_sd = ', np.sum(np.power(trigger_between_40_60_Serum_Calcium_mean_not_nan, 2)))
print('trigger_above_60_Serum_Calcium_mean_sd = ', np.sum(np.power(trigger_above_60_Serum_Calcium_mean_not_nan, 2)))

In [ ]:
# Urine_Protein_Creatinine_pandas

count_list = list(patients_vs_LOINC_lab_procedures['patient_sk'])
item_list = list(patients_vs_LOINC_lab_procedures['Urine_Protein_Creatinine'])

trigger_below_20 = list(set(count_list) & set(trigger_below_20))
trigger_between_20_40 = list(set(count_list) & set(trigger_between_20_40))
trigger_between_40_60 = list(set(count_list) & set(trigger_between_40_60))
trigger_above_60 = list(set(count_list) & set(trigger_above_60))

trigger_below_20_Urine_Protein_Creatinine_mean = []
trigger_between_20_40_Urine_Protein_Creatinine_mean = []
trigger_between_40_60_Urine_Protein_Creatinine_mean = []
trigger_above_60_Urine_Protein_Creatinine_mean = []

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        indexx = count_list.index(patient_sk)
        trigger_below_20_Urine_Protein_Creatinine_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_20_40:
        indexx = count_list.index(patient_sk)
        trigger_between_20_40_Urine_Protein_Creatinine_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_40_60:
        indexx = count_list.index(patient_sk)
        trigger_between_40_60_Urine_Protein_Creatinine_mean.append(item_list[indexx])
    elif patient_sk in trigger_above_60:
        indexx = count_list.index(patient_sk)
        trigger_above_60_Urine_Protein_Creatinine_mean.append(item_list[indexx])

print('trigger_below_20_Urine_Protein_Creatinine_mean = ', np.nanmean(trigger_below_20_Urine_Protein_Creatinine_mean))
print('trigger_between_20_40_Urine_Protein_Creatinine_mean = ', np.nanmean(trigger_between_20_40_Urine_Protein_Creatinine_mean))
print('trigger_between_40_60_Urine_Protein_Creatinine_mean = ', np.nanmean(trigger_between_40_60_Urine_Protein_Creatinine_mean))
print('trigger_above_60_Urine_Protein_Creatinine_mean = ', np.nanmean(trigger_above_60_Urine_Protein_Creatinine_mean))



trigger_below_20_Urine_Protein_Creatinine_mean_not_nan = [x for x in trigger_below_20_Urine_Protein_Creatinine_mean if np.isnan(x)==False]
trigger_between_20_40_Urine_Protein_Creatinine_mean_not_nan = [x for x in trigger_between_20_40_Urine_Protein_Creatinine_mean if np.isnan(x)==False]
trigger_between_40_60_Urine_Protein_Creatinine_mean_not_nan = [x for x in trigger_between_40_60_Urine_Protein_Creatinine_mean if np.isnan(x)==False]
trigger_above_60_Urine_Protein_Creatinine_mean_not_nan = [x for x in trigger_above_60_Urine_Protein_Creatinine_mean if np.isnan(x)==False]

print('trigger_below_20_Urine_Protein_Creatinine_mean = ', np.nanmean(trigger_below_20_Urine_Protein_Creatinine_mean_not_nan))
print('trigger_between_20_40_Urine_Protein_Creatinine_mean_mean = ', np.nanmean(trigger_between_20_40_Urine_Protein_Creatinine_mean_not_nan))
print('trigger_between_40_60_Urine_Protein_Creatinine_mean = ', np.nanmean(trigger_between_40_60_Urine_Protein_Creatinine_mean_not_nan))
print('trigger_above_60_Urine_Protein_Creatinine_mean = ', np.nanmean(trigger_above_60_Urine_Protein_Creatinine_mean_not_nan))

print('trigger_below_20_Urine_Protein_Creatinine_mean_count = ', len(trigger_below_20_Urine_Protein_Creatinine_mean_not_nan))
print('trigger_between_20_40_Urine_Protein_Creatinine_mean_count = ', len(trigger_between_20_40_Urine_Protein_Creatinine_mean_not_nan))
print('trigger_between_40_60_Urine_Protein_Creatinine_mean_count = ', len(trigger_between_40_60_Urine_Protein_Creatinine_mean_not_nan))
print('trigger_above_60_Urine_Protein_Creatinine_mean_count = ', len(trigger_above_60_Urine_Protein_Creatinine_mean_not_nan))

print('trigger_below_20_Urine_Protein_Creatinine_mean_sd = ', np.sum(np.power(trigger_below_20_Urine_Protein_Creatinine_mean_not_nan, 2)))
print('trigger_between_20_40_Urine_Protein_Creatinine_mean_sd = ', np.sum(np.power(trigger_between_20_40_Urine_Protein_Creatinine_mean_not_nan, 2)))
print('trigger_between_40_60_Urine_Protein_Creatinine_mean_sd = ', np.sum(np.power(trigger_between_40_60_Urine_Protein_Creatinine_mean_not_nan, 2)))
print('trigger_above_60_Urine_Protein_Creatinine_mean_sd = ', np.sum(np.power(trigger_above_60_Urine_Protein_Creatinine_mean_not_nan, 2)))

In [ ]:
# twentyfour_hr_Urine_Protein_pandas

count_list = list(patients_vs_LOINC_lab_procedures['patient_sk'])
item_list = list(patients_vs_LOINC_lab_procedures['24_hr_Urine_Protein'])

trigger_below_20 = list(set(count_list) & set(trigger_below_20))
trigger_between_20_40 = list(set(count_list) & set(trigger_between_20_40))
trigger_between_40_60 = list(set(count_list) & set(trigger_between_40_60))
trigger_above_60 = list(set(count_list) & set(trigger_above_60))

trigger_below_20_twentyfour_hr_Urine_Protein_mean = []
trigger_between_20_40_twentyfour_hr_Urine_Protein_mean = []
trigger_between_40_60_twentyfour_hr_Urine_Protein_mean = []
trigger_above_60_twentyfour_hr_Urine_Protein_mean = []

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        indexx = count_list.index(patient_sk)
        trigger_below_20_twentyfour_hr_Urine_Protein_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_20_40:
        indexx = count_list.index(patient_sk)
        trigger_between_20_40_twentyfour_hr_Urine_Protein_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_40_60:
        indexx = count_list.index(patient_sk)
        trigger_between_40_60_twentyfour_hr_Urine_Protein_mean.append(item_list[indexx])
    elif patient_sk in trigger_above_60:
        indexx = count_list.index(patient_sk)
        trigger_above_60_twentyfour_hr_Urine_Protein_mean.append(item_list[indexx])

print('trigger_below_20_twentyfour_hr_Urine_Protein_mean = ', np.nanmean(trigger_below_20_twentyfour_hr_Urine_Protein_mean))
print('trigger_between_20_40_twentyfour_hr_Urine_Protein_mean = ', np.nanmean(trigger_between_20_40_twentyfour_hr_Urine_Protein_mean))
print('trigger_between_40_60_twentyfour_hr_Urine_Protein_mean = ', np.nanmean(trigger_between_40_60_twentyfour_hr_Urine_Protein_mean))
print('trigger_above_60_twentyfour_hr_Urine_Protein_mean = ', np.nanmean(trigger_above_60_twentyfour_hr_Urine_Protein_mean))


trigger_below_20_twentyfour_hr_Urine_Protein_mean_not_nan = [x for x in trigger_below_20_twentyfour_hr_Urine_Protein_mean if np.isnan(x)==False]
trigger_between_20_40_twentyfour_hr_Urine_Protein_mean_not_nan = [x for x in trigger_between_20_40_twentyfour_hr_Urine_Protein_mean if np.isnan(x)==False]
trigger_between_40_60_twentyfour_hr_Urine_Protein_mean_not_nan = [x for x in trigger_between_40_60_twentyfour_hr_Urine_Protein_mean if np.isnan(x)==False]
trigger_above_60_twentyfour_hr_Urine_Protein_mean_not_nan = [x for x in trigger_above_60_twentyfour_hr_Urine_Protein_mean if np.isnan(x)==False]

print('trigger_below_20_twentyfour_hr_Urine_Protein_mean = ', np.nanmean(trigger_below_20_twentyfour_hr_Urine_Protein_mean_not_nan))
print('trigger_between_20_40_twentyfour_hr_Urine_Protein_mean = ', np.nanmean(trigger_between_20_40_twentyfour_hr_Urine_Protein_mean_not_nan))
print('trigger_between_40_60_twentyfour_hr_Urine_Protein_mean = ', np.nanmean(trigger_between_40_60_twentyfour_hr_Urine_Protein_mean_not_nan))
print('trigger_above_60_twentyfour_hr_Urine_Protein_mean = ', np.nanmean(trigger_above_60_twentyfour_hr_Urine_Protein_mean_not_nan))

print('trigger_below_20_twentyfour_hr_Urine_Protein_mean_count = ', len(trigger_below_20_twentyfour_hr_Urine_Protein_mean_not_nan))
print('trigger_between_20_40_twentyfour_hr_Urine_Protein_mean_count = ', len(trigger_between_20_40_twentyfour_hr_Urine_Protein_mean_not_nan))
print('trigger_between_40_60_twentyfour_hr_Urine_Protein_mean_count = ', len(trigger_between_40_60_twentyfour_hr_Urine_Protein_mean_not_nan))
print('trigger_above_60_twentyfour_hr_Urine_Protein_mean_count = ', len(trigger_above_60_twentyfour_hr_Urine_Protein_mean_not_nan))

print('trigger_below_20_twentyfour_hr_Urine_Protein_mean_sd = ', np.sum(np.power(trigger_below_20_twentyfour_hr_Urine_Protein_mean_not_nan, 2)))
print('trigger_between_20_40_twentyfour_hr_Urine_Protein_mean_sd = ', np.sum(np.power(trigger_between_20_40_twentyfour_hr_Urine_Protein_mean_not_nan, 2)))
print('trigger_between_40_60_twentyfour_hr_Urine_Protein_mean_sd = ', np.sum(np.power(trigger_between_40_60_twentyfour_hr_Urine_Protein_mean_not_nan, 2)))
print('trigger_above_60_twentyfour_hr_Urine_Protein_mean_sd = ', np.sum(np.power(trigger_above_60_twentyfour_hr_Urine_Protein_mean_not_nan, 2)))

In [ ]:
# Hemoglobin_A1c

count_list = list(patients_vs_LOINC_lab_procedures['patient_sk'])
item_list = list(patients_vs_LOINC_lab_procedures['Hemoglobin_A1c'])
        
trigger_below_20 = list(set(count_list) & set(trigger_below_20))
trigger_between_20_40 = list(set(count_list) & set(trigger_between_20_40))
trigger_between_40_60 = list(set(count_list) & set(trigger_between_40_60))
trigger_above_60 = list(set(count_list) & set(trigger_above_60))

trigger_below_20_Hemoglobin_A1c_mean = []
trigger_between_20_40_Hemoglobin_A1c_mean = []
trigger_between_40_60_Hemoglobin_A1c_mean = []
trigger_above_60_Hemoglobin_A1c_mean = []

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        indexx = count_list.index(patient_sk)
        trigger_below_20_Hemoglobin_A1c_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_20_40:
        indexx = count_list.index(patient_sk)
        trigger_between_20_40_Hemoglobin_A1c_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_40_60:
        indexx = count_list.index(patient_sk)
        trigger_between_40_60_Hemoglobin_A1c_mean.append(item_list[indexx])
    elif patient_sk in trigger_above_60:
        indexx = count_list.index(patient_sk)
        trigger_above_60_Hemoglobin_A1c_mean.append(item_list[indexx])

print('trigger_below_20_Hemoglobin_A1c_mean = ', np.nanmean(trigger_below_20_Hemoglobin_A1c_mean))
print('trigger_between_20_40_Hemoglobin_A1c_mean = ', np.nanmean(trigger_between_20_40_Hemoglobin_A1c_mean))
print('trigger_between_40_60_Hemoglobin_A1c_mean = ', np.nanmean(trigger_between_40_60_Hemoglobin_A1c_mean))
print('trigger_above_60_Hemoglobin_A1c_mean = ', np.nanmean(trigger_above_60_Hemoglobin_A1c_mean))


trigger_below_20_Hemoglobin_A1c_mean_not_nan = [x for x in trigger_below_20_Hemoglobin_A1c_mean if np.isnan(x)==False]
trigger_between_20_40_Hemoglobin_A1c_mean_not_nan = [x for x in trigger_between_20_40_Hemoglobin_A1c_mean if np.isnan(x)==False]
trigger_between_40_60_Hemoglobin_A1c_mean_not_nan = [x for x in trigger_between_40_60_Hemoglobin_A1c_mean if np.isnan(x)==False]
trigger_above_60_Hemoglobin_A1c_mean_not_nan = [x for x in trigger_above_60_Hemoglobin_A1c_mean if np.isnan(x)==False]

print('trigger_below_20_Hemoglobin_A1c_mean = ', np.nanmean(trigger_below_20_Hemoglobin_A1c_mean_not_nan))
print('trigger_between_20_40_Hemoglobin_A1c_mean_mean = ', np.nanmean(trigger_between_20_40_Hemoglobin_A1c_mean_not_nan))
print('trigger_between_40_60_Hemoglobin_A1c_mean_mean = ', np.nanmean(trigger_between_40_60_Hemoglobin_A1c_mean_not_nan))
print('trigger_above_60_Hemoglobin_A1c_mean_mean = ', np.nanmean(trigger_above_60_Hemoglobin_A1c_mean_not_nan))

print('trigger_below_20_Hemoglobin_A1c_mean_count = ', len(trigger_below_20_Hemoglobin_A1c_mean_not_nan))
print('trigger_between_20_40_Hemoglobin_A1c_mean_count = ', len(trigger_between_20_40_Hemoglobin_A1c_mean_not_nan))
print('trigger_between_40_60_Hemoglobin_A1c_mean_count = ', len(trigger_between_40_60_Hemoglobin_A1c_mean_not_nan))
print('trigger_above_60_Hemoglobin_A1c_mean_count = ', len(trigger_above_60_Hemoglobin_A1c_mean_not_nan))

print('trigger_below_20_Hemoglobin_A1c_mean_sd = ', np.sum(np.power(trigger_below_20_Hemoglobin_A1c_mean_not_nan, 2)))
print('trigger_between_20_40_Hemoglobin_A1c_mean_sd = ', np.sum(np.power(trigger_between_20_40_Hemoglobin_A1c_mean_not_nan, 2)))
print('trigger_between_40_60_Hemoglobin_A1c_mean_sd = ', np.sum(np.power(trigger_between_40_60_Hemoglobin_A1c_mean_not_nan, 2)))
print('trigger_above_60_Hemoglobin_A1c_mean_sd = ', np.sum(np.power(trigger_above_60_Hemoglobin_A1c_mean_not_nan, 2)))

In [ ]:
# Hemoglobin

count_list = list(patients_vs_LOINC_lab_procedures['patient_sk'])
item_list = list(patients_vs_LOINC_lab_procedures['Hemoglobin'])

trigger_below_20 = list(set(count_list) & set(trigger_below_20))
trigger_between_20_40 = list(set(count_list) & set(trigger_between_20_40))
trigger_between_40_60 = list(set(count_list) & set(trigger_between_40_60))
trigger_above_60 = list(set(count_list) & set(trigger_above_60))

trigger_below_20_Hemoglobin_mean = []
trigger_between_20_40_Hemoglobin_mean = []
trigger_between_40_60_Hemoglobin_mean = []
trigger_above_60_Hemoglobin_mean = []

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        indexx = count_list.index(patient_sk)
        trigger_below_20_Hemoglobin_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_20_40:
        indexx = count_list.index(patient_sk)
        trigger_between_20_40_Hemoglobin_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_40_60:
        indexx = count_list.index(patient_sk)
        trigger_between_40_60_Hemoglobin_mean.append(item_list[indexx])
    elif patient_sk in trigger_above_60:
        indexx = count_list.index(patient_sk)
        trigger_above_60_Hemoglobin_mean.append(item_list[indexx])

print('trigger_below_20_Hemoglobin_mean = ', np.nanmean(trigger_below_20_Hemoglobin_mean))
print('trigger_between_20_40_Hemoglobin_mean = ', np.nanmean(trigger_between_20_40_Hemoglobin_mean))
print('trigger_between_40_60_Hemoglobin_mean = ', np.nanmean(trigger_between_40_60_Hemoglobin_mean))
print('trigger_above_60_Hemoglobin_mean = ', np.nanmean(trigger_above_60_Hemoglobin_mean))


trigger_below_20_Hemoglobin_mean_not_nan = [x for x in trigger_below_20_Hemoglobin_mean if np.isnan(x)==False]
trigger_between_20_40_Hemoglobin_mean_not_nan = [x for x in trigger_between_20_40_Hemoglobin_mean if np.isnan(x)==False]
trigger_between_40_60_Hemoglobin_mean_not_nan = [x for x in trigger_between_40_60_Hemoglobin_mean if np.isnan(x)==False]
trigger_above_60_Hemoglobin_mean_not_nan = [x for x in trigger_above_60_Hemoglobin_mean if np.isnan(x)==False]

print('trigger_below_20_Hemoglobin_mean = ', np.nanmean(trigger_below_20_Hemoglobin_mean_not_nan))
print('trigger_between_20_40_Hemoglobin_mean = ', np.nanmean(trigger_between_20_40_Hemoglobin_mean_not_nan))
print('trigger_between_40_60_Hemoglobin_mean = ', np.nanmean(trigger_between_40_60_Hemoglobin_mean_not_nan))
print('trigger_above_60_Hemoglobin_mean = ', np.nanmean(trigger_above_60_Hemoglobin_mean_not_nan))

print('trigger_below_20_Hemoglobin_mean_count = ', len(trigger_below_20_Hemoglobin_mean_not_nan))
print('trigger_between_20_40_Hemoglobin_mean_count = ', len(trigger_between_20_40_Hemoglobin_mean_not_nan))
print('trigger_between_40_60_Hemoglobin_mean_count = ', len(trigger_between_40_60_Hemoglobin_mean_not_nan))
print('trigger_above_60_Hemoglobin_mean_count = ', len(trigger_above_60_Hemoglobin_mean_not_nan))

print('trigger_below_20_Hemoglobin_mean_sd = ', np.sum(np.power(trigger_below_20_Hemoglobin_mean_not_nan, 2)))
print('trigger_between_20_40_Hemoglobin_mean_sd = ', np.sum(np.power(trigger_between_20_40_Hemoglobin_mean_not_nan, 2)))
print('trigger_between_40_60_Hemoglobin_mean_sd = ', np.sum(np.power(trigger_between_40_60_Hemoglobin_mean_not_nan, 2)))
print('trigger_above_60_Hemoglobin_mean_sd = ', np.sum(np.power(trigger_above_60_Hemoglobin_mean_not_nan, 2)))

In [ ]:
# Hepatitis_C

count_list = list(patients_vs_LOINC_lab_procedures['patient_sk'])
item_list = list(patients_vs_LOINC_lab_procedures['Hepatitis_C'])

for i in range(len(item_list)):
    if pd.isnull(item_list[i]) == True:
        item_list[i] = 0  

trigger_below_20 = list(set(count_list) & set(trigger_below_20))
trigger_between_20_40 = list(set(count_list) & set(trigger_between_20_40))
trigger_between_40_60 = list(set(count_list) & set(trigger_between_40_60))
trigger_above_60 = list(set(count_list) & set(trigger_above_60))

trigger_below_20_Hepatitis_C_count = 0
trigger_between_20_40_Hepatitis_C_count = 0
trigger_between_40_60_Hepatitis_C_count = 0
trigger_above_60_Hepatitis_C_count = 0

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        trigger_below_20_Hepatitis_C_count = trigger_below_20_Hepatitis_C_count + (item_list[count_list.index(patient_sk)])
    elif patient_sk in trigger_between_20_40:
        trigger_between_20_40_Hepatitis_C_count = trigger_between_20_40_Hepatitis_C_count + (item_list[count_list.index(patient_sk)])    
    elif patient_sk in trigger_between_40_60:
        trigger_between_40_60_Hepatitis_C_count = trigger_between_40_60_Hepatitis_C_count + (item_list[count_list.index(patient_sk)])
    elif patient_sk in trigger_above_60:
        trigger_above_60_Hepatitis_C_count = trigger_above_60_Hepatitis_C_count + (item_list[count_list.index(patient_sk)])

print('trigger_below_20_Hepatitis_C_mean = ', trigger_below_20_Hepatitis_C_count)
print('trigger_between_20_40_Hepatitis_C_mean = ', trigger_between_20_40_Hepatitis_C_count)
print('trigger_between_40_60_Hepatitis_C_mean = ', trigger_between_40_60_Hepatitis_C_count)
print('trigger_above_60_Hepatitis_C_mean = ', trigger_above_60_Hepatitis_C_count)

In [ ]:
# Serum_Cholesterol

count_list = list(patients_vs_LOINC_lab_procedures['patient_sk'])
item_list = list(patients_vs_LOINC_lab_procedures['Serum_Cholesterol'])

trigger_below_20 = list(set(count_list) & set(trigger_below_20))
trigger_between_20_40 = list(set(count_list) & set(trigger_between_20_40))
trigger_between_40_60 = list(set(count_list) & set(trigger_between_40_60))
trigger_above_60 = list(set(count_list) & set(trigger_above_60))

trigger_below_20_Serum_Cholesterol_mean = []
trigger_between_20_40_Serum_Cholesterol_mean = []
trigger_between_40_60_Serum_Cholesterol_mean = []
trigger_above_60_Serum_Cholesterol_mean = []

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        indexx = count_list.index(patient_sk)
        trigger_below_20_Serum_Cholesterol_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_20_40:
        indexx = count_list.index(patient_sk)
        trigger_between_20_40_Serum_Cholesterol_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_40_60:
        indexx = count_list.index(patient_sk)
        trigger_between_40_60_Serum_Cholesterol_mean.append(item_list[indexx])
    elif patient_sk in trigger_above_60:
        indexx = count_list.index(patient_sk)
        trigger_above_60_Serum_Cholesterol_mean.append(item_list[indexx])

print('trigger_below_20_Serum_Cholesterol_mean = ', np.nanmean(trigger_below_20_Serum_Cholesterol_mean))
print('trigger_between_20_40_Serum_Cholesterol_mean = ', np.nanmean(trigger_between_20_40_Serum_Cholesterol_mean))
print('trigger_between_40_60_Serum_Cholesterol_mean = ', np.nanmean(trigger_between_40_60_Serum_Cholesterol_mean))
print('trigger_above_60_Serum_Cholesterol_mean = ', np.nanmean(trigger_above_60_Serum_Cholesterol_mean))


trigger_below_20_Serum_Cholesterol_mean_not_nan = [x for x in trigger_below_20_Serum_Cholesterol_mean if np.isnan(x)==False]
trigger_between_20_40_Serum_Cholesterol_mean_not_nan = [x for x in trigger_between_20_40_Serum_Cholesterol_mean if np.isnan(x)==False]
trigger_between_40_60_Serum_Cholesterol_mean_not_nan = [x for x in trigger_between_40_60_Serum_Cholesterol_mean if np.isnan(x)==False]
trigger_above_60_Serum_Cholesterol_mean_not_nan = [x for x in trigger_above_60_Serum_Cholesterol_mean if np.isnan(x)==False]

print('trigger_below_20_Serum_Cholesterol_mean_mean = ', np.nanmean(trigger_below_20_Serum_Cholesterol_mean_not_nan))
print('trigger_between_20_40_Serum_Cholesterol_mean_mean = ', np.nanmean(trigger_between_20_40_Serum_Cholesterol_mean_not_nan))
print('trigger_between_40_60_Serum_Cholesterol_mean_mean = ', np.nanmean(trigger_between_40_60_Serum_Cholesterol_mean_not_nan))
print('trigger_above_60_Serum_Cholesterol_mean_mean = ', np.nanmean(trigger_above_60_Serum_Cholesterol_mean_not_nan))

print('trigger_below_20_Serum_Cholesterol_mean_count = ', len(trigger_below_20_Serum_Cholesterol_mean_not_nan))
print('trigger_between_20_40_Serum_Cholesterol_mean_count = ', len(trigger_between_20_40_Serum_Cholesterol_mean_not_nan))
print('trigger_between_40_60_Serum_Cholesterol_mean_count = ', len(trigger_between_40_60_Serum_Cholesterol_mean_not_nan))
print('trigger_above_60_Serum_Cholesterol_mean_count = ', len(trigger_above_60_Serum_Cholesterol_mean_not_nan))

print('trigger_below_20_Serum_Cholesterol_mean_sd = ', np.sum(np.power(trigger_below_20_Serum_Cholesterol_mean_not_nan, 2)))
print('trigger_between_20_40_Serum_Cholesterol_mean_sd = ', np.sum(np.power(trigger_between_20_40_Serum_Cholesterol_mean_not_nan, 2)))
print('trigger_between_40_60_Serum_Cholesterol_mean_sd = ', np.sum(np.power(trigger_between_40_60_Serum_Cholesterol_mean_not_nan, 2)))
print('trigger_above_60_Serum_Cholesterol_mean_sd = ', np.sum(np.power(trigger_above_60_Serum_Cholesterol_mean_not_nan, 2)))

In [ ]:
# Serum_albumin

count_list = list(patients_vs_LOINC_lab_procedures['patient_sk'])
item_list = list(patients_vs_LOINC_lab_procedures['Serum_albumin'])

zz = len(item_list)
i = 0
for item in item_list:
    if np.isnan(item) == False:
        if (item >= 100):
            zz = zz - 1
            item_list[i] = np.nan
    i = i + 1
        
        
trigger_below_20 = list(set(count_list) & set(trigger_below_20))
trigger_between_20_40 = list(set(count_list) & set(trigger_between_20_40))
trigger_between_40_60 = list(set(count_list) & set(trigger_between_40_60))
trigger_above_60 = list(set(count_list) & set(trigger_above_60))

trigger_below_20_Serum_albumin_mean = []
trigger_between_20_40_Serum_albumin_mean = []
trigger_between_40_60_Serum_albumin_mean = []
trigger_above_60_Serum_albumin_mean = []

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        indexx = count_list.index(patient_sk)
        trigger_below_20_Serum_albumin_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_20_40:
        indexx = count_list.index(patient_sk)
        trigger_between_20_40_Serum_albumin_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_40_60:
        indexx = count_list.index(patient_sk)
        trigger_between_40_60_Serum_albumin_mean.append(item_list[indexx])
    elif patient_sk in trigger_above_60:
        indexx = count_list.index(patient_sk)
        trigger_above_60_Serum_albumin_mean.append(item_list[indexx])

print('trigger_below_20_Serum_albumin_mean = ', np.nanmean(trigger_below_20_Serum_albumin_mean))
print('trigger_between_20_40_Serum_albumin_mean = ', np.nanmean(trigger_between_20_40_Serum_albumin_mean))
print('trigger_between_40_60_Serum_albumin_mean = ', np.nanmean(trigger_between_40_60_Serum_albumin_mean))
print('trigger_above_60_Serum_albumin_mean = ', np.nanmean(trigger_above_60_Serum_albumin_mean))


trigger_below_20_Serum_albumin_mean_not_nan = [x for x in trigger_below_20_Serum_albumin_mean if np.isnan(x)==False]
trigger_between_20_40_Serum_albumin_mean_not_nan = [x for x in trigger_between_20_40_Serum_albumin_mean if np.isnan(x)==False]
trigger_between_40_60_Serum_albumin_mean_not_nan = [x for x in trigger_between_40_60_Serum_albumin_mean if np.isnan(x)==False]
trigger_above_60_Serum_albumin_mean_not_nan = [x for x in trigger_above_60_Serum_albumin_mean if np.isnan(x)==False]

print('trigger_below_20_Serum_albumin_mean_mean = ', np.nanmean(trigger_below_20_Serum_albumin_mean_not_nan))
print('trigger_between_20_40_Serum_albumin_mean_mean = ', np.nanmean(trigger_between_20_40_Serum_albumin_mean_not_nan))
print('trigger_between_40_60_Serum_albumin_mean_mean = ', np.nanmean(trigger_between_40_60_Serum_albumin_mean_not_nan))
print('trigger_above_60_Serum_albumin_mean_mean = ', np.nanmean(trigger_above_60_Serum_albumin_mean_not_nan))

print('trigger_below_20_Serum_albumin_mean_count = ', len(trigger_below_20_Serum_albumin_mean_not_nan))
print('trigger_between_20_40_Serum_albumin_mean_count = ', len(trigger_between_20_40_Serum_albumin_mean_not_nan))
print('trigger_between_40_60_Serum_albumin_mean_count = ', len(trigger_between_40_60_Serum_albumin_mean_not_nan))
print('trigger_above_60_Serum_albumin_mean_count = ', len(trigger_above_60_Serum_albumin_mean_not_nan))

print('trigger_below_20_Serum_albumin_mean_sd = ', np.sum(np.power(trigger_below_20_Serum_albumin_mean_not_nan, 2)))
print('trigger_between_20_40_Serum_albumin_mean_sd = ', np.sum(np.power(trigger_between_20_40_Serum_albumin_mean_not_nan, 2)))
print('trigger_between_40_60_Serum_albumin_mean_sd = ', np.sum(np.power(trigger_between_40_60_Serum_albumin_mean_not_nan, 2)))
print('trigger_above_60_Serum_albumin_mean_sd = ', np.sum(np.power(trigger_above_60_Serum_albumin_mean_not_nan, 2)))

In [ ]:
# Serum_Phosphorus

count_list = list(patients_vs_LOINC_lab_procedures['patient_sk'])
item_list = list(patients_vs_LOINC_lab_procedures['Serum_Phosphorus'])

trigger_below_20 = list(set(count_list) & set(trigger_below_20))
trigger_between_20_40 = list(set(count_list) & set(trigger_between_20_40))
trigger_between_40_60 = list(set(count_list) & set(trigger_between_40_60))
trigger_above_60 = list(set(count_list) & set(trigger_above_60))

trigger_below_20_Serum_Phosphorus_mean = []
trigger_between_20_40_Serum_Phosphorus_mean = []
trigger_between_40_60_Serum_Phosphorus_mean = []
trigger_above_60_Serum_Phosphorus_mean = []

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        indexx = count_list.index(patient_sk)
        trigger_below_20_Serum_Phosphorus_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_20_40:
        indexx = count_list.index(patient_sk)
        trigger_between_20_40_Serum_Phosphorus_mean.append(item_list[indexx])
    elif patient_sk in trigger_between_40_60:
        indexx = count_list.index(patient_sk)
        trigger_between_40_60_Serum_Phosphorus_mean.append(item_list[indexx])
    elif patient_sk in trigger_above_60:
        indexx = count_list.index(patient_sk)
        trigger_above_60_Serum_Phosphorus_mean.append(item_list[indexx])

print('trigger_below_20_Serum_Phosphorus_mean = ', np.nanmean(trigger_below_20_Serum_Phosphorus_mean))
print('trigger_between_20_40_Serum_Phosphorus_mean = ', np.nanmean(trigger_between_20_40_Serum_Phosphorus_mean))
print('trigger_between_40_60_Serum_Phosphorus_mean = ', np.nanmean(trigger_between_40_60_Serum_Phosphorus_mean))
print('trigger_above_60_Serum_Phosphorus_mean = ', np.nanmean(trigger_above_60_Serum_Phosphorus_mean))


trigger_below_20_Serum_Phosphorus_mean_not_nan = [x for x in trigger_below_20_Serum_Phosphorus_mean if np.isnan(x)==False]
trigger_between_20_40_Serum_Phosphorus_mean_not_nan = [x for x in trigger_between_20_40_Serum_Phosphorus_mean if np.isnan(x)==False]
trigger_between_40_60_Serum_Phosphorus_mean_not_nan = [x for x in trigger_between_40_60_Serum_Phosphorus_mean if np.isnan(x)==False]
trigger_above_60_Serum_Phosphorus_mean_not_nan = [x for x in trigger_above_60_Serum_Phosphorus_mean if np.isnan(x)==False]

print('trigger_below_20_Serum_Phosphorus_mean_mean = ', np.nanmean(trigger_below_20_Serum_Phosphorus_mean_not_nan))
print('trigger_between_20_40_Serum_Phosphorus_mean_mean = ', np.nanmean(trigger_between_20_40_Serum_Phosphorus_mean_not_nan))
print('trigger_between_40_60_Serum_Phosphorus_mean_mean = ', np.nanmean(trigger_between_40_60_Serum_Phosphorus_mean_not_nan))
print('trigger_above_60_Serum_Phosphorus_mean_mean = ', np.nanmean(trigger_above_60_Serum_Phosphorus_mean_not_nan))

print('trigger_below_20_Serum_Phosphorus_mean_count = ', len(trigger_below_20_Serum_Phosphorus_mean_not_nan))
print('trigger_between_20_40_Serum_Phosphorus_mean_count = ', len(trigger_between_20_40_Serum_Phosphorus_mean_not_nan))
print('trigger_between_40_60_Serum_Phosphorus_mean_count = ', len(trigger_between_40_60_Serum_Phosphorus_mean_not_nan))
print('trigger_above_60_Serum_Phosphorus_mean_count = ', len(trigger_above_60_Serum_Phosphorus_mean_not_nan))

print('trigger_below_20_Serum_Phosphorus_mean_sd = ', np.sum(np.power(trigger_below_20_Serum_Phosphorus_mean_not_nan, 2)))
print('trigger_between_20_40_Serum_Phosphorus_mean_sd = ', np.sum(np.power(trigger_between_20_40_Serum_Phosphorus_mean_not_nan, 2)))
print('trigger_between_40_60_Serum_Phosphorus_mean_sd = ', np.sum(np.power(trigger_between_40_60_Serum_Phosphorus_mean_not_nan, 2)))
print('trigger_above_60_Serum_Phosphorus_mean_sd = ', np.sum(np.power(trigger_above_60_Serum_Phosphorus_mean_not_nan, 2)))

In [ ]:
# Medication

In [ ]:
# NSAID

count_list = list(patients_vs_medications['patient_sk'])
item_list = list(patients_vs_medications['NSAID'])

for i in range(len(item_list)):
    if pd.isnull(item_list[i]) == True:
        item_list[i] = 0  

trigger_below_20 = list(set(count_list) & set(trigger_below_20))
trigger_between_20_40 = list(set(count_list) & set(trigger_between_20_40))
trigger_between_40_60 = list(set(count_list) & set(trigger_between_40_60))
trigger_above_60 = list(set(count_list) & set(trigger_above_60))

trigger_below_20_NSAID_count = 0
trigger_between_20_40_NSAID_count = 0
trigger_between_40_60_NSAID_count = 0
trigger_above_60_NSAID_count = 0

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        trigger_below_20_NSAID_count = trigger_below_20_NSAID_count + (item_list[count_list.index(patient_sk)])
    elif patient_sk in trigger_between_20_40:
        trigger_between_20_40_NSAID_count = trigger_between_20_40_NSAID_count + (item_list[count_list.index(patient_sk)])    
    elif patient_sk in trigger_between_40_60:
        trigger_between_40_60_NSAID_count = trigger_between_40_60_NSAID_count + (item_list[count_list.index(patient_sk)])
    elif patient_sk in trigger_above_60:
        trigger_above_60_NSAID_count = trigger_above_60_NSAID_count + (item_list[count_list.index(patient_sk)])

print('trigger_below_20_NSAID_count = ', trigger_below_20_NSAID_count)
print('trigger_between_20_40_NSAID_count = ', trigger_between_20_40_NSAID_count)
print('trigger_between_40_60_NSAID_count = ', trigger_between_40_60_NSAID_count)
print('trigger_above_60_NSAID_count = ', trigger_above_60_NSAID_count)

In [ ]:
# Proton_Pump_Inhibitors_pandas 

count_list = list(patients_vs_medications['patient_sk'])
item_list = list(patients_vs_medications['Proton_Pump_Inhibitors'])

for i in range(len(item_list)):
    if pd.isnull(item_list[i]) == True:
        item_list[i] = 0  

trigger_below_20 = list(set(count_list) & set(trigger_below_20))
trigger_between_20_40 = list(set(count_list) & set(trigger_between_20_40))
trigger_between_40_60 = list(set(count_list) & set(trigger_between_40_60))
trigger_above_60 = list(set(count_list) & set(trigger_above_60))

trigger_below_20_Proton_Pump_Inhibitors_count = 0
trigger_between_20_40_Proton_Pump_Inhibitors_count = 0
trigger_between_40_60_Proton_Pump_Inhibitors_count = 0
trigger_above_60_Proton_Pump_Inhibitors_count = 0

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        trigger_below_20_Proton_Pump_Inhibitors_count = trigger_below_20_Proton_Pump_Inhibitors_count + (item_list[count_list.index(patient_sk)])
    elif patient_sk in trigger_between_20_40:
        trigger_between_20_40_Proton_Pump_Inhibitors_count = trigger_between_20_40_Proton_Pump_Inhibitors_count + (item_list[count_list.index(patient_sk)])    
    elif patient_sk in trigger_between_40_60:
        trigger_between_40_60_Proton_Pump_Inhibitors_count = trigger_between_40_60_Proton_Pump_Inhibitors_count + (item_list[count_list.index(patient_sk)])
    elif patient_sk in trigger_above_60:
        trigger_above_60_Proton_Pump_Inhibitors_count = trigger_above_60_Proton_Pump_Inhibitors_count + (item_list[count_list.index(patient_sk)])

print('trigger_below_20_Proton_Pump_Inhibitors_count = ', trigger_below_20_Proton_Pump_Inhibitors_count)
print('trigger_between_20_40_Proton_Pump_Inhibitors_count = ', trigger_between_20_40_Proton_Pump_Inhibitors_count)
print('trigger_between_40_60_Proton_Pump_Inhibitors_count = ', trigger_between_40_60_Proton_Pump_Inhibitors_count)
print('trigger_above_60_Proton_Pump_Inhibitors_count = ', trigger_above_60_Proton_Pump_Inhibitors_count)

In [ ]:
# Bipolar_Drugs_pandas

count_list = list(patients_vs_medications['patient_sk'])
item_list = list(patients_vs_medications['Bipolar_Drugs'])

for i in range(len(item_list)):
    if pd.isnull(item_list[i]) == True:
        item_list[i] = 0  

trigger_below_20 = list(set(count_list) & set(trigger_below_20))
trigger_between_20_40 = list(set(count_list) & set(trigger_between_20_40))
trigger_between_40_60 = list(set(count_list) & set(trigger_between_40_60))
trigger_above_60 = list(set(count_list) & set(trigger_above_60))

trigger_below_20_Bipolar_Drugs_count = 0
trigger_between_20_40_Bipolar_Drugs_count = 0
trigger_between_40_60_Bipolar_Drugs_count = 0
trigger_above_60_Bipolar_Drugs_count = 0

for patient_sk in count_list:
    if patient_sk in trigger_below_20:
        trigger_below_20_Bipolar_Drugs_count = trigger_below_20_Bipolar_Drugs_count + (item_list[count_list.index(patient_sk)])
    elif patient_sk in trigger_between_20_40:
        trigger_between_20_40_Bipolar_Drugs_count = trigger_between_20_40_Bipolar_Drugs_count + (item_list[count_list.index(patient_sk)])    
    elif patient_sk in trigger_between_40_60:
        trigger_between_40_60_Bipolar_Drugs_count = trigger_between_40_60_Bipolar_Drugs_count + (item_list[count_list.index(patient_sk)])
    elif patient_sk in trigger_above_60:
        trigger_above_60_Bipolar_Drugs_count = trigger_above_60_Bipolar_Drugs_count + (item_list[count_list.index(patient_sk)])

print('trigger_below_20_Bipolar_Drugs_count = ', trigger_below_20_Bipolar_Drugs_count)
print('trigger_between_20_40_Bipolar_Drugs_count = ', trigger_between_20_40_Bipolar_Drugs_count)
print('trigger_between_40_60_Bipolar_Drugs_count = ', trigger_between_40_60_Bipolar_Drugs_count)
print('trigger_above_60_Bipolar_Drugs_count = ', trigger_above_60_Bipolar_Drugs_count)

### For further information please contact rzz5164@psu.edu